# Extinction

Define extinction as
$$
A_\lambda = +2.5\log\left( \frac{F_{\lambda 0}}{F_\lambda} \right)
$$

So, the extinction function can be interpreted as extinguished flux by
$$
F_\lambda = F_{\lambda 0} 10^{-0.4A_\lambda}.
$$

More interesting is to then convert magnitudes with the filter functions,
$$
\Delta m = -2.5\log\left(\int W_\lambda F_\lambda d\lambda\right) + 2.5\log\left(\int W_\lambda F_{\lambda 0} d\lambda \right)
$$
Are the zero points the same for the original flux and the extinguished flux?  Yes.  But the point here is that it is impossible to correctly calculate the magnitude extinction without the orginal flux,
$$
\Delta m = -2.5\log\left(\int W_\lambda F_{\lambda 0} 10^{-0.4A_\lambda} d\lambda\right) + 2.5\log\left(\int W_\lambda F_{\lambda 0} d\lambda \right)
$$

Conclusion?  If and only if the filter efficiency is constant in the band and the flux is constant in the band can one convert a "theory extinction curve" into magnitudes:
$$
\Delta m = -2.5\log\left(\frac{\int 10^{-0.4A_\lambda} d\lambda}{\int d\lambda}\right)
$$

As a shorthand, the average of $A_\lambda$ in the band is the magnitude extinction.  In general, however, the band response and the source flux must be taken into consideration.